In [1]:
import pandas as pd

In [4]:
nl_2008_2017 = pd.read_csv('nl_2008_2017.csv')
nl_2008_2017.shape

(155, 19)

In [5]:
nl_2008_2017.head()

,Team,Year,Wins,TeamWAR,AllP,SP,RP,Non-P,C,1B,2B,3B,SS,LF,CF,RF,OF (All),DH,PH
0,ARI,2008.0,82.0,30.1,22.7,18.1,5.0,7.4,1.2,0.2,2.1,0.7,3.2,-0.6,1.4,0.0,0.8,0.0,-0.8
1,ATL,2008.0,72.0,28.4,4.3,4.4,-0.1,24.1,4.1,4.1,2.6,7.6,3.3,0.3,2.2,-2.0,0.5,0.2,1.7
2,CHC,2008.0,97.0,49.0,29.1,22.9,5.6,19.9,2.8,1.9,3.3,3.0,2.2,2.4,1.6,0.9,4.9,0.1,1.7
3,CIN,2008.0,74.0,21.0,12.0,6.8,6.1,9.0,0.3,2.7,2.8,0.4,-1.1,2.6,0.5,0.6,3.7,0.0,0.2
4,COL,2008.0,74.0,26.6,13.0,8.2,6.5,13.6,2.4,1.0,1.7,1.2,1.8,5.8,-0.4,-0.2,5.2,0.0,0.3


In [7]:
nl_2008_2017

,Team,Year,Wins,TeamWAR,AllP,SP,RP,Non-P,C,1B,2B,3B,SS,LF,CF,RF,OF (All),DH,PH
0,ARI,2008.0,82.0,30.1,22.7,18.1,5.0,7.4,1.2,0.2,2.1,0.7,3.2,-0.6,1.4,0.0,0.8,0.0,-0.8
1,ATL,2008.0,72.0,28.4,4.3,4.4,-0.1,24.1,4.1,4.1,2.6,7.6,3.3,0.3,2.2,-2.0,0.5,0.2,1.7
2,CHC,2008.0,97.0,49.0,29.1,22.9,5.6,19.9,2.8,1.9,3.3,3.0,2.2,2.4,1.6,0.9,4.9,0.1,1.7
3,CIN,2008.0,74.0,21.0,12.0,6.8,6.1,9.0,0.3,2.7,2.8,0.4,-1.1,2.6,0.5,0.6,3.7,0.0,0.2
4,COL,2008.0,74.0,26.6,13.0,8.2,6.5,13.6,2.4,1.0,1.7,1.2,1.8,5.8,-0.4,-0.2,5.2,0.0,0.3
5,FLA,2008.0,84.0,28.6,9.2,6.5,3.7,19.4,1.3,-1.2,4.6,1.1,6.9,2.5,3.5,0.1,6.1,0.0,0.6
6,HOU,2008.0,86.0,27.1,10.8,6.6,4.0,16.3,-0.7,6.7,1.3,1.8,1.9,3.1,-0.5,2.3,4.9,0.3,0.1
7,LAD,2008.0,84.0,36.4,20.9,13.3,8.0,15.5,3.6,0.7,-0.2,2.2,1.9,3.9,0.8,2.9,7.6,-0.2,-0.1
8,MIL,2008.0,90.0,41.4,15.2,13.2,1.9,26.2,3.2,1.7,2.7,1.6,5.1,4.7,4.0,1.6,10.3,0.1,1.5
9,NYM,2008.0,89.0,39.2,14.1,13.0,1.2,25.1,0.7,1.3,-0.9,6.8,4.8,2.3,6.9,2.2,11.4,0.1,0.9
